In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893



Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [4]:
import tensorflow as tf
# Enforce dynamic session for Keras. Run to avoid GPU ram hogging
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [5]:
# Original Sequential, fast and reliable
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [11]:
import random
import sys
from IPython.display import clear_output

for epoch in range(1, 10):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y, batch_size=128, epochs=1)
    # clear output to prevent clutter
    clear_output()

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]: #
        print('------ constant temperature:', temperature)
        sys.stdout.write(generated_text)
        output_constant_temperature(generated_text, model, num_gen=400, temperature=temperature)
    for decay_fraction in [2.5, 2.0, 1.5]:
        print('------ temperature that perk at separator (%.1f) and decay %.1f per char:' % (1.0, decay_fraction))
        sys.stdout.write(generated_text)
        output_perk_temperature(generated_text, model, num_gen=400, max_temp=1.0, min_temp=0.1,
                                    decay_fn=lambda x: x/decay_fraction)
    for threshold in [0.5, 0.55, 0.6]: #
        print('------ loaned temperature threshold:', threshold)
        sys.stdout.write(generated_text)
        output_loan_temperature(generated_text, model, num_gen=400, willful_threshold=threshold)
    

--- Generating with seed: "thinks and
lives gangasrotogati [footnote: like the river ga"
------ constant temperature: 0.2
thinks and
lives gangasrotogati [footnote: like the river gain the reade and the readed to the reading the extent and the readed and the ready to the conscience the readed and the seeped to the extent and the master and the reade of a man is the reade of the readous to the read and the sense to the extent and the perhaps the reade the man is a man and the sense the extent and in the religious and the instinction of the one and the peristence and the period
Average maximum preds:  0.4879219
------ constant temperature: 0.5
thinks and
lives gangasrotogati [footnote: like the river gain and trate as which is not of a sceated meriation, it is feeling and the respect to the restent feeling and to the readistance and the ralstion of the refunces even and the invent and the expects to his in enter and is of the posted to the repposition of the religious expect of the ready

KeyboardInterrupt: 

In [8]:
def output_constant_temperature(seed_text, model, num_gen=400, temperature=1.0):
    # constant temperature throughout the text
    maxpreds = []
    for i in range(num_gen):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        seed_text += next_char
        seed_text = seed_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
        maxpreds.append(np.max(preds))
    print() #print("\nAverage maximum preds: ", np.mean(maxpreds))

In [9]:
def output_perk_temperature(seed_text, model, num_gen=400, max_temp=1.0, min_temp=0.1, decay_fn=lambda x: x/2.0):
    # customized temperature, strongest when a separator (not a character) is encountered and sharply decay over time
    temperature = min_temp if seed_text[-1].isalpha() else max_temp
    for i in range(num_gen):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        temperature = max_temp if not next_char.isalpha() else max(decay_fn(temperature), min_temp)
        seed_text += next_char
        seed_text = seed_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [17]:
#import math
def output_loan_temperature(seed_text, model, num_gen=400, willful_threshold=0.5):
    # loan temperature, calculate how 'willful' the last character was, and subtract/add it to the current temperature
    temperature = 0.5
    for i in range(num_gen):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        temperature += (preds[next_index] - willful_threshold) * 0.01
        seed_text += next_char
        seed_text = seed_text[1:]

        sys.stdout.write(next_char) # "%s(%.1f)" % (next_char, temperature))
        sys.stdout.flush()
    print()
    
sys.stdout.write(generated_text)
output_loan_temperature(generated_text, model, num_gen=400, willful_threshold=0.4)

thinks and
lives gangasrotogati [footnote: like the river ganity of the respect in the pares to such to the our in the reased the he we feele religious dened the postently to that it is lece presented of the intentialy of the one sensely that is secied, the respect of the vive as late feell the germany the news experious sheakent bece the moraling instreir and there out the eones to be to to ence out in the sen one seepless to
speists and the repant to the



As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in 
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text 
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as 
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings 
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment 
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and 
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is 
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is 
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To 
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like 
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic 
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.